# Movie genres 

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import joblib

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/naumovskiy/HW4/main/HW4/made_hw_2/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/naumovskiy/HW4/main/HW4/made_hw_2/test.csv')
treshold_df = pd.read_csv('https://raw.githubusercontent.com/naumovskiy/HW4/main/HW4/made_hw_2/treshold_df.csv',index_col='index')

In [3]:
mlb = MultiLabelBinarizer()
train.genres = train.genres.apply(eval)
y = pd.DataFrame(mlb.fit_transform(train.genres), columns=mlb.classes_)

In [4]:
vectorizer = TfidfVectorizer(min_df=5)
vectorizer.fit(train.dialogue.append(test.dialogue))

X_train = vectorizer.transform(train.dialogue)
y_train = y.copy()
ovrc = OneVsRestClassifier(LogisticRegression(C=10), n_jobs=-1)
ovrc.fit(X_train, y_train);

### TO APP

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
with open('/content/drive/My Drive/data/movie_vectorizer_dump.pkl', 'wb') as output_file:
    joblib.dump(vectorizer, output_file)

with open('/content/drive/My Drive/data/movie_model_dump.pkl', 'wb') as output_file:
    joblib.dump(ovrc, output_file)

with open('/content/drive/My Drive/data/movie_mlb_dump.pkl', 'wb') as output_file:
    joblib.dump(mlb, output_file)    

with open('/content/drive/My Drive/data/movie_treshold_df_dump.pkl', 'wb') as output_file:
    joblib.dump(treshold_df, output_file)      

### IN APP

In [8]:
text_ = "war and world"

In [9]:
preds = ovrc.predict_proba(vectorizer.transform([text_]))
preds

array([[9.57921177e-01, 6.71282263e-01, 4.75196948e-03, 2.65343657e-01,
        2.81236025e-01, 8.59372958e-04, 3.04348611e-01, 8.02318371e-03,
        1.31308581e-02, 6.51042865e-03, 1.53149129e-04, 2.51508517e-04,
        1.38098482e-03, 3.24386400e-02, 2.08348723e-02, 2.19759532e-01,
        9.57784751e-04, 2.98155998e-04, 3.04966941e-01, 8.08373619e-04]])

In [10]:
for n, yi in enumerate(['action', 'adventure', 'animation', 'biography', 'comedy', 'crime',
                        'drama', 'family', 'fantasy', 'history', 'horror', 'music', 'musical',
                        'mystery', 'romance', 'sci-fi', 'sport', 'thriller', 'war', 'western']):
    preds[:, n] = preds[:, n] > treshold_df.loc[yi, 'best_treshold']

In [11]:
if preds.sum() == 0:
  preds[6] = 1

In [12]:
' '.join(mlb.inverse_transform(preds)[0])

'action adventure biography war'

In [13]:
mlb.inverse_transform(preds)[0]

('action', 'adventure', 'biography', 'war')